In [21]:
import datetime
from pyspark import SparkConf, SparkContext, HiveContext
sc.stop()
conf = SparkConf().set("spark.executor.instances", 1).set("spark.driver.maxResultSize", "1g")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)
#sc.setCheckpointDir('/user/kposminin/checkpointdir/')

In [23]:
create_result_table_query = '''


create table user_kposminin.full_app_vs_approve as
 select
    uf.id,
    max(if(ua.id is Null,0,1)) as approve_label,
    max(nvl(uf.first_day,0)) as full_app_first_day,
    max(log((t1.cnt_positive + 0.1)/(t2.cnt_positive - t1.cnt_positive + 0.1))) as max_score,
    avg(log((t1.cnt_positive + 0.1)/(t2.cnt_positive - t1.cnt_positive + 0.1))) as avg_score,
    '2016-11-24' as ymd
  from
    (
      select distinct id, if(ymd = '2016-11-25',1,0) as first_day
      from prod_features_liveinternet.user_action
      where action_type = 'tinkoff_platinum_complete_application'
        and ymd between '2016-11-25' and '2016-11-27'
    ) uf
    left join(
      select distinct id, if(ymd = '2016-11-25',1,0) as first_day
      from prod_features_liveinternet.user_action
      where action_type = 'tinkoff_platinum_approved_application'
        and ymd between '2016-11-25' and '2016-11-27'
    ) ua on uf.id = ua.id
  inner join prod_features_liveinternet.visits v on uf.id = v.id 
  left join (
    select
      urlfr, 
      cnt_positive,
      cnt_total
    from prod_features_liveinternet.urlfr_tgt_cnt
    where
      ymd = '2016-11-23' and
      target = 'tinkoff_platinum_approved_application@tinkoff_action' and
      (cnt_total > 10000 or cnt_positive >= 1)
    ) t1 on t1.urlfr = v.urlfr
  left join (
    select
      urlfr, 
      cnt_positive,
      cnt_total
    from prod_features_liveinternet.urlfr_tgt_cnt
    where
      ymd = '2016-11-23' and
      target = 'tinkoff_platinum_complete_application@tinkoff_action' and
      (cnt_total > 10000 or cnt_positive > 1)
    ) t2 on t1.urlfr = v.urlfr
where
  v.ymd = '2016-11-24'
group by uf.id    
;

insert into table user_kposminin.full_app_vs_approve
 select
    uf.id,
    max(if(ua.id is Null,0,1)) as approve_label,
    max(nvl(uf.first_day,0)) as full_app_first_day,
    max(log((t1.cnt_positive + 0.1)/(t2.cnt_positive - t1.cnt_positive + 0.1))) as max_score,
    avg(log((t1.cnt_positive + 0.1)/(t2.cnt_positive - t1.cnt_positive + 0.1))) as avg_score,
    '2016-11-15' as ymd
  from
    (
      select distinct id, if(ymd = '2016-11-16',1,0) as first_day
      from prod_features_liveinternet.user_action
      where action_type = 'tinkoff_platinum_complete_application'
        and ymd between '2016-11-16' and '2016-11-18'
    ) uf
    left join(
      select distinct id, if(ymd = '2016-11-16',1,0) as first_day
      from prod_features_liveinternet.user_action
      where action_type = 'tinkoff_platinum_approved_application'
        and ymd between '2016-11-16' and '2016-11-18'
    ) ua on uf.id = ua.id
  inner join prod_features_liveinternet.visits v on uf.id = v.id 
  left join (
    select
      urlfr, 
      cnt_positive,
      cnt_total
    from prod_features_liveinternet.urlfr_tgt_cnt
    where
      ymd = '2016-11-14' and
      target = 'tinkoff_platinum_approved_application@tinkoff_action' and
      (cnt_total > 10000 or cnt_positive >= 1)
    ) t1 on t1.urlfr = v.urlfr
  left join (
    select
      urlfr, 
      cnt_positive,
      cnt_total
    from prod_features_liveinternet.urlfr_tgt_cnt
    where
      ymd = '2016-11-14' and
      target = 'tinkoff_platinum_complete_application@tinkoff_action' and
      (cnt_total > 10000 or cnt_positive > 1)
    ) t2 on t1.urlfr = v.urlfr
where
  v.ymd = '2016-11-15'
group by uf.id    
;


'''


df = hc.sql('''select approve_label,full_app_first_day,nvl(max_score,-20) as max_score, nvl(avg_score,-20) as avg_score
               from user_kposminin.full_app_vs_approve''').toPandas()
df.head()

,approve_label,full_app_first_day,max_score,avg_score
0,1,0,4.709530,-1.317073
1,0,1,7.378384,-1.194880
2,1,1,7.378384,0.126864
3,0,1,6.792344,-0.762199
4,0,1,6.647688,-2.002536


In [28]:
#df[df.in_work==1].full_app.sum()
#data = sorted(df[['avg_score','approve_label']].values.tolist(), reverse = True)
df['approve_first_day'] = df.approve_label * df.full_app_first_day

In [49]:
from sklearn.metrics import roc_auc_score,precision_recall_curve,auc
   
print('Max score AUC ROC: {}'.format(roc_auc_score(y_true = df.approve_label,y_score = df.max_score)))
print('Avg score AUC ROC: {}'.format(roc_auc_score(y_true = df.approve_label,y_score = df.avg_score)))
print('Max score first_day. AUC ROC: {}'.format(roc_auc_score(y_true = df.approve_first_day,y_score = df.max_score)))
print('Avg score  first_day. AUC ROC: {}'.format(roc_auc_score(y_true = df.approve_first_day,y_score = df.avg_score)))

print('\nMax score AUC PR: {}'.format(auc(*precision_recall_curve(y_true = df.approve_label,probas_pred  = df.max_score)[:2],reorder = True)))
print('Avg score AUC PR: {}'.format(auc(*precision_recall_curve(y_true = df.approve_label,probas_pred  = df.avg_score)[:2],reorder = True)))
print('Max score first_day. AUC PR: {}'.format(auc(*precision_recall_curve(y_true = df.approve_first_day,probas_pred  = df.max_score)[:2],reorder = True)))
print('Avg score  first_day. AUC PR: {}'.format(auc(*precision_recall_curve(y_true = df.approve_first_day,probas_pred  = df.avg_score)[:2],reorder = True)))


Max score AUC ROC: 0.532380423374
Avg score AUC ROC: 0.511590682577
Max score first_day. AUC ROC: 0.547225533054
Avg score  first_day. AUC ROC: 0.513407424055

Max score AUC PR: 0.061704826149
Avg score AUC PR: 0.0106046428652
Max score first_day. AUC PR: 0.0757242517789
Avg score  first_day. AUC PR: 0.00576490014429


In [61]:
data = sorted(df[['max_score','approve_label']].values.tolist(), reverse = True)
data 
def nvl(a,b):
    if a:
        return a
    else:
        return b
    
print('\nLift table for ph_stand_la\n\n{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}'.format(
        'q','pos_cnt','pos%','cnt','pos/cnt','lift','score'))
pos_tot = sum([e[1] for e in data])
pos_share_tot = float(pos_tot) / len(data)
for q in [0.95,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0]:
    pos_cnt = sum([e[1] for e in data[:int((1-q)*len(data))]])
    cnt = int((1-q)*len(data))
    pos_share = float(pos_cnt)/cnt
    print('{:<10}\t{:<10}\t{:<10.2%}\t{:<10}\t{:<10.5%}\t{:<10.2f}\t{:<10.5f}'.format(q,
                                                                                      pos_cnt,
                                                                                      float(pos_cnt)/pos_tot,
                                                                                      cnt,pos_share,
                                                                                      pos_share/pos_share_tot,
                                                                                      nvl(data[int((1-q)*len(data))-1][0],-20)
                                                                                     ))


Lift table for ph_stand_la

q         	pos_cnt   	pos%      	cnt       	pos/cnt   	lift      	score     
0.95      	186.0     	10.51%    	857       	21.70362% 	2.10      	7.37838   
0.9       	565.0     	31.92%    	1714      	32.96383% 	3.19      	7.35308   
0.8       	565.0     	31.92%    	3429      	16.47711% 	1.60      	7.35308   
0.7       	669.0     	37.80%    	5144      	13.00544% 	1.26      	6.87833   
0.6       	766.0     	43.28%    	6859      	11.16781% 	1.08      	6.58064   
0.5       	980.0     	55.37%    	8574      	11.42990% 	1.11      	6.04263   
0.4       	1148.0    	64.86%    	10289     	11.15755% 	1.08      	5.60212   
0.3       	1288.0    	72.77%    	12004     	10.72976% 	1.04      	4.88898   
0.2       	1465.0    	82.77%    	13719     	10.67862% 	1.03      	3.93183   
0         	1770.0    	100.00%   	17149     	10.32130% 	1.00      	-20.00000 


### Классификатор на основе стандартной модели работает плохо. Нужен более тонкий подход.